#### DNN 기반 회귀 모델 구현 + 학습 진행 모니터링 + 진행 파라미터 저장
- 사용되는 데이터셋 : iris.csv
- feature : 3개
- target : 1개 
- 학습방법 : 지도학습 -> 회귀
- 알고리즘 : 인공신경망(ANN) -> 심층(은닉층) 신경망 -> MLP(층이여러개), DNN(은닉층이 많은 구성) 
- FramWork : Pytorch  

#### - 모니터링
    - 기준 설정 : 검증데이터셋의 loss와 score
    - 평가 설정 : 학습데이터셋의 loss와 score와 비교해서 학습 중단 여부를 결정
    - 선택 설정 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장

#### - 진행 파라미터 저장


[1] 모듈  로딩 & 데이터  준비 <HR>

In [2]:
# 모듈 로딩
# 모델관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 전처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split



In [3]:
# 활용 패키지 버전 체크
def versioncheck():
    print(f' torch  {torch.__version__}')
    print(f' pandas  {pd.__version__}')

versioncheck()

 torch  2.4.1
 pandas  2.0.3


In [4]:
DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE, usecols=[0,1,2,3])
irisdf

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


[2] - 모델 클레스 설계 및 정의 - <hr>
어떤 클래스를 만들까? 고려
클래스 목적 : iris.DataSet 학습 후 추론
클래스 이름 : IrisRegModel
부모 클래스 : nn.Module
매개 변수   : 층별 입출력 갯수 고정하기 때문에 필요 없음
클래스 속성 : featureDF, targetDF, n_rows, n_features
클래스 기능 : __init__() : 모델 구조, forward() : 순방향 학습 <= 오버라이딩(overriding) 상속관계일 때

클래스 구조  
    -입력층 : 피쳐 3개  퍼셉트론 : 10개(보통 입력 때 많이 주고 갈수록 줄임) (3,10)
    -은닉층 : 입력 10개     출력 30개   (10,30)
    -출력층 : 입력30개      타겟 1개    (30,1)  
  
-손실 함수/ 활성화 함수
    *클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드에 사용
    *함수 형태 ==> torch.nn.functional => forward()메서드에 사용

In [5]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(3, 10)
        self.hidden_layer=nn.Linear(10, 30)
        self.out_layer=nn.Linear(30, 1)
    
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y=self.in_layer(input_data) # f1w1+f2w2+f3w3+b 요런 식이 10개(숫자10개)
        y=F.relu(y)                   # 범위 0이상
        
        # 은닉층 : 10개의 숫자 받아오기
        y=self.hidden_layer(y)
        y=F.relu(y)

        # 출력층 : 30개의 숫자 값
        return self.out_layer(y)
    






In [6]:
# 모델 인스턴스 생성
model = IrisRegModel()
print(model)

# 모델 사용 메모리 정보
summary(model, input_size=(1000,3))

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1000, 1]                 --
├─Linear: 1-1                            [1000, 10]                40
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 1]                 31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.40
Input size (MB): 0.01
Forward/backward pass size (MB): 0.33
Params size (MB): 0.00
Estimated Total Size (MB): 0.34

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataSet
- 부모클래스 : utils.data.DataSet
- 속성__필드 : featureDF, targetDF, n_rows, n_featrues  
- 필수 메서드:   
    *__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정  
    *__len__(self) : 데이터의 개수 반환  
    *__getItem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환  

In [7]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [8]:
# 데이터셋 인스턴스 생성
featureDF = irisdf[irisdf.columns[:-1]] # 2D (150,3)
targetDF = irisdf[irisdf.columns[-1:]] # 1D(150,1)


irisDS=IrisDataset(featureDF,targetDF)


[4] 학습 준비
- 학습 횟수 : EPOCH ( 처음부터 끝까지 공부할 횟수 )
- 배치 크기 : BATCH_SIZE(학습량)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 GPU/CPU)
- 학 습 률  : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1
- 

In [9]:
# 학습 진행 관련 설정 값
EPOCH = 10
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001


- 인스턴스 : 모델, 데이터 셋, 최적화 (, 성능지표) 

In [10]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')


trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 데이터 로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

# 최적화

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)


In [11]:
## 테스트
for feature, target in trainDL:
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [12]:

# 최적화 인스턴스 => w, b model.parameter 전달
optimizer=optim.Adam(model.parameters(), lr=LR)


# 손실함수 인스턴스
reqLoss=nn.MSELoss()

[5] 학습 진행

In [21]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일을 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/'

# 저장 파일명
SAVE_FILE = 'model_train_wbs.pth'

# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더 / 폴더 / ... 하위 폴더까지 생성

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL='model_all.pth'

In [22]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT = irisDS.n_rows / BATCH_SIZE
print(f'CNT => {CNT}')

# 학습 모니터링 / 스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT = 0


# 학습 모드로 모델 설정
model.train()

for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0

    for featureTS, targetTS in trainDL:

        #학습 진행
        pre_y=model(featureTS)

        #손실 계산
        loss=reqLoss(pre_y, targetTS)
        loss_total += loss.item()


        #성능평가 계산
        score=R2Score()(pre_y, targetTS)
        score_total += score.item()

        #최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에폭당 검증 기능을 키겠다.
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        #평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqLoss(pre_val, val_targetTS)

        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)

    # 에폭당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 학습 진행 모니터링 / 스케쥴링
    # # 손실 기준
    # if len(LOSS_HISTORY[0]) > 1:
    #    if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2] : BREAK_CNT +=1
    # 성능이 좋아진걸 다 파일로 담고 싶을 때 SAVE_FILE = f'model_train_wbs_{epoch}_{score_val}.pth'


    # 성능 기준
    if len(SCORE_HISTORY[1]) == 1: # 첫번째 횟수 저장
       torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE)

       # 모델 전체 저장
       torch.save(model, SAVE_PATH + SAVE_MODEL)
    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]) : # 첫번째 점수랑 두번째 점수 비교 후 더 성능이 큰쪽을 저장
            torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE)
            torch.save(model, SAVE_PATH + SAVE_FILE)

    # 성능이 좋은 학습 가중치 저장

    
    # 학습중단
    if BREAK_CNT >10:
        print('성능 및 손실 개선이 없어서 학습 중단') 
        break



CNT => 15.0


In [13]:
print(LOSS_HISTORY)
print(SCORE_HISTORY)

[[4.622160577774048, 2.8437670469284058, 1.6159888982772828, 0.8612407445907593, 0.4825149983167648, 0.3434994637966156, 0.3045129060745239, 0.2776761993765831, 0.24658086895942688, 0.21839893758296966], [tensor(3.3580), tensor(1.9429), tensor(1.0418), tensor(0.5873), tensor(0.4351), tensor(0.4172), tensor(0.4025), tensor(0.3575), tensor(0.3045), tensor(0.2594)]]
[[-9.084650373458862, -5.277884459495544, -2.630307745933533, -0.9816407799720764, -0.13346655964851378, 0.193197500705719, 0.2886703312397003, 0.3442302465438843, 0.4038241982460022, 0.4571122109889984], [tensor(-4.2021), tensor(-2.0099), tensor(-0.6139), tensor(0.0901), tensor(0.3259), tensor(0.3537), tensor(0.3764), tensor(0.4462), tensor(0.5283), tensor(0.5981)]]


- 모델 저장 방법 <hr>
* 방법 1 : 모델 파라미터만 저장
* 방법 2 : 모델 설계 구조 및 파라미터까지 모두 저장


In [16]:
# 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[ 0.4294, -0.1866, -0.3250],
                      [-0.2630,  0.1907, -0.2610],
                      [ 0.3991,  0.0131,  0.2289],
                      [ 0.1383, -0.3683,  0.3524],
                      [-0.5571, -0.5640, -0.2533],
                      [-0.5369, -0.0483, -0.2632],
                      [-0.0092, -0.4191, -0.3201],
                      [ 0.5375, -0.1915,  0.4072],
                      [-0.5737, -0.2079, -0.0431],
                      [ 0.4088,  0.5008, -0.4124]])),
             ('in_layer.bias',
              tensor([ 0.2869, -0.2863, -0.1278, -0.3487,  0.2373, -0.3904,  0.1875, -0.4133,
                       0.4756,  0.1231])),
             ('hidden_layer.weight',
              tensor([[ 1.3026e-01, -2.2578e-02,  2.4019e-02, -9.4844e-02, -2.8551e-01,
                       -2.4399e-02,  4.9848e-02,  1.3181e-01, -2.7374e-01,  1.4921e-01],
                      [-2.7451e-02, -4.4807e-02,  9.8158e-02, -3.5809e-0

- 방법1 : 모델 파라미터 즉, 층별 가중치와 절편들

In [17]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일을 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/'

# 저장 파일명
SAVE_FILE = 'model_train_wbs.pth'

In [20]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더 / 폴더 / ... 하위 폴더까지 생성


In [23]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE)

In [27]:
# 모델 즉, 가중치와 절편 로딩
# 1. 가중치와 절편 객체로 로딩
# 2. 모델의 state_dict 속성에 저장

# 읽기
wbTS=torch.load( SAVE_PATH + SAVE_FILE)

# 모델 인스턴스에 저장
model2 = IrisRegModel()
model2.load_state_dict(wbTS)

C:\Users\KDP-26\AppData\Local\Temp\ipykernel_12524\1308290621.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wbTS=torch.load( SAVE_PATH + SAVE_FILE)


<All keys matched successfully>